# Programming Hidden Markov Models (60 P)

In this exercise, you will experiment with hidden Markov models, in particular, applying them to modeling character sequences, and analyzing the learned solution. As a starting point, you are provided in the file `hmm.py` with a basic implementation of an HMM and of the Baum-Welch training algorithm. The names of variables used in the code and the references to equations are taken from the HMM paper by Rabiner et al. downloable from ISIS. In addition to the variables described in this paper, we use two additional variables: $Z$ for the emission probabilities of observations $O$, and $\psi$ (i.e. psi) for collecting the statistics of Equation (40c).


## Question 1: Analysis of a small HMM (30 P)

We first look at a toy example of an HMM trained on a binary sequence. The training procedure below consists of 100 iterations of the Baum-Welch procedure. It runs the HMM learning algorithm for some toy binary data and prints the parameters learned by the HMM (i.e. matrices $A$ and $B$).

### Question 1a: Qualitative Analysis (15 P)

* *Run* the code several times to check that the behavior is consistent.
* *Describe* qualitatively the solution $A,B$ learned by the model.
* *Explain* how the solution $\lambda = (A,B)$ relates to the sequence of observations $O$ that has been modeled.


In [14]:
import numpy
import hmm

O = numpy.array([1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
                 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
                 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
                 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
                 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
                 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
                 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0])

hmmtoy = hmm.HMM(4, 2)

for k in range(100):
    hmmtoy.loaddata(O)
    hmmtoy.forward()
    hmmtoy.backward()
    hmmtoy.learn()

print('A')
print("\n".join([" ".join(['%.3f' % a for a in aa]) for aa in hmmtoy.A]))
print(' ')
print('B')
print("\n".join([" ".join(['%.3f' % b for b in bb]) for bb in hmmtoy.B]))
print(' ')
print('Pi')
print("\n".join(['%.3f' % b for b in hmmtoy.Pi]))

A
0.000 0.000 1.000 0.000
0.000 0.000 0.000 1.000
0.000 1.000 0.000 0.000
1.000 0.000 0.000 0.000
 
B
0.720 0.280
0.880 0.120
0.800 0.200
0.000 1.000
 
Pi
0.000
0.000
0.000
1.000


### Question 1b: Finding the best number $N$ of hidden states (15 P)

For the same sequence of observations as in Question 1a, we would like to determine automatically what is a good number of hidden states $N = \mathrm{card}(S)$ for the model.

* *Split* the sequence of observations into a training and test set (you can assume stationarity).
* *Train* the model on the training set for several iteration (e.g. 100 iterations) and for multiple parameter $N$.
* *Show* for each choice of parameter $N$ the log-probability $\log p(O | \lambda)$ for the test set. (If the results are unstable, perform several trials of the same experiment for each parameter $N$.)
* *Explain* in the light of this experiment what is the best parameter $N$.

In [15]:
for N in [2, 4, 8, 16]:
    print(f'N = {N}')
    hmmtoy = hmm.HMM(N, 2)

    Otrain = O[:len(O)//2]
    Otest = O[len(O)//2:]

    for k in range(100):
        hmmtoy.loaddata(Otrain)
        hmmtoy.forward()
        hmmtoy.backward()
        hmmtoy.learn()

    hmmtoy.loaddata(Otrain)
    hmmtoy.forward()
    print(f'log train error = {numpy.log(hmmtoy.pobs)}')

    hmmtoy.loaddata(Otest)
    hmmtoy.forward()
    print(f'test train error = {numpy.log(hmmtoy.pobs)}')

N = 2
log train error = -56.2405840801898
test train error = -61.57473563854022
N = 4
log train error = -37.77395592017892
test train error = -36.30146850469229
N = 8
log train error = -35.82569631431537
test train error = -62.418379294918815
N = 16
log train error = -30.443708236881474
test train error = -221.64177281958214


## Question 2: Text modeling and generation (30 P)

We would like to train an HMM on character sequences taken from English text. We use the 20 newsgroups dataset that is accessible via scikits-learn http://scikit-learn.org/stable/datasets/twenty_newsgroups.html. (For this, you need to install scikits-learn if not done already.) Documentation is available on the website. The code below allows you to (1) read the dataset, (2) sample HMM-readable sequences from it, and (3) convert them back into string of characters.

In [16]:
from sklearn.datasets import fetch_20newsgroups

# Download a subset of the newsgroup dataset
newsgroups_train = fetch_20newsgroups(subset='train', categories=['sci.med'])
newsgroups_test = fetch_20newsgroups(subset='test', categories=['sci.med'])

# Sample a sequence of T characters from the dataset
# that the HMM can read (0=whitespace 1-26=A-Z).
#
# Example of execution:
# O = sample(newsgroups_train.data)
# O = sample(newsgroups_test.data)
#


def sample(data, T=50):
    i = numpy.random.randint(len(data))
    O = data[i].upper().replace('\n', ' ')
    O = numpy.array([ord(s) for s in O])
    O = numpy.maximum(O[(O >= 65)*(O < 90)+(O == 32)]-64, 0)
    j = numpy.random.randint(len(O)-T)
    return O[j:j+T]

# Takes a sequence of integers between 0 and 26 (HMM representation)
# and converts it back to a string of characters


def tochar(O):
    return "".join(["%s" % chr(o) for o in (O+32*(O == 0)+64*(O > 0.5))])

### Question 2a (15 P)

In order to train the HMM, we use a stochastic optimization algorithm where the Baum-Welch procedure is applied to randomly drawn sequences of $T=50$ characters at each iteration. The HMM has 27 visible states (A-Z + whitespace) and 200 hidden states. Because the Baum-Welch procedure optimizes for the sequence taken as input, and no necessarily the full text, it can take fairly large steps in the parameter space, which is inadequate for stochastic optimization. We consider instead for the parameters $\lambda = (A,B,\Pi)$ the update rule $\lambda^{new} = (1-\gamma) \lambda + \gamma \bar \lambda$, where $\bar \lambda$ contains the candidate parameters obtained from Equations 40a-c. A reasonable value for $\gamma$ is $0.1$.

* *Create* a new class `HMMChar` that extends the class `HMM` provided in `hmm.py`.
* *Implement* for this class a new method `HMMchar.learn(self)` that overrides the original methods, and implements the proposed update rule instead.
* *Implement* the stochastic training procedure and run it.
* *Monitor* $\log p(O|\lambda)$ on the test set at multiple iterations for sequences of same length as the one used for training. (Hint: for less noisy log-probability estimates, use several sequences or a moving average.)

In [17]:
na = numpy.newaxis


class HMMChar(hmm.HMM):
    def learn(self):
        # Compute gamma
        self.gamma = self.alpha*self.beta / self.pobs

        # Compute xi and psi
        self.xi = self.alpha[:-1, :, na]*self.A[na, :, :] * \
            self.beta[1:, na, :]*self.Z[1:, na, :] / self.pobs
        self.psi = self.gamma[:, :, na] * \
            (self.O[:, na, na] == numpy.arange(self.B.shape[1])[na, na, :])

        # Update HMM parameters
        self.A = 0.9*self.A + 0.1 * \
            (self.xi.sum(axis=0) / self.gamma[:-1].sum(axis=0)[:, na])
        self.B = 0.9*self.B + 0.1 * \
            (self.psi.sum(axis=0) / self.gamma.sum(axis=0)[:, na])
        self.Pi = 0.9*self.Pi + 0.1*(self.gamma[0])

    def generate(self, l):
        N, M = len(self.Pi), self.B.shape[1]

        s = numpy.random.choice(N, p=self.Pi)
        O = []
        for i in range(l):
            O += [numpy.random.choice(M, p=self.B[s])]
            s = numpy.random.choice(N, p=self.A[s])

        return numpy.array(O)

In [18]:

hmmchar = HMMChar(200, 27)
def trainsample(): return sample(newsgroups_train.data)


def testsample(): return sample(newsgroups_test.data)


pobstrain = []
pobstest = []

for k in range(1000):
    Otrain = trainsample()
    Otest = testsample()

    hmmchar.loaddata(Otrain)
    hmmchar.forward()
    hmmchar.backward()
    hmmchar.learn()

    hmmchar.loaddata(Otrain)
    hmmchar.forward()
    pobstrain += [hmmchar.pobs]

    hmmchar.loaddata(Otest)
    hmmchar.forward()
    pobstest += [hmmchar.pobs]

    if k % 100 == 0:
        print(k)
        print(f'train error = {numpy.mean(numpy.log(pobstrain))}')
        print(f'test error = {numpy.mean(numpy.log(pobstest))}')

0
train error = -156.29777807436508
test error = -158.3311801937664
100
train error = -139.08630070924931
test error = -141.9351345214084
200
train error = -131.95010317395707
test error = -136.39902351100272
300
train error = -126.47201959018878
test error = -132.53219891174498
400
train error = -122.50984733970628
test error = -130.48189574642547
500
train error = -120.42168477281326
test error = -129.04222264771792
600
train error = -118.60588099358023
test error = -128.02603659423792
700
train error = -117.75526936359199
test error = -127.31488687247604
800
train error = -116.8347284058903
test error = -126.97346845999573
900
train error = -116.18462796731873
test error = -126.45916327277148


### Question 2b (15 P)

In order to visualize what the HMM has learned, we would like to generate random text from it. A well-trained HMM should generate character sequences that have some similarity with the text it has been trained on.

* *Implement* a method `generate(self,T)` of the class `HMMChar` that takes as argument the length of the character sequence that has to be generated.
* *Test* your method by generating a sequence of 250 characters and comparing it with original text and a purely random sequence.
* *Discuss* how the generated sequences compare with written English and what are the advantages and limitations of the HMM for this problem.

In [19]:
print("original:\n"+tochar(sample(newsgroups_test.data, T=250)))
print("\nlearned:\n"+tochar(hmmchar.generate(250)))
print("\nrandom:\n" + tochar(HMMChar(200, 27).generate(250)))

original:
AY ID LIKE FEEDBACK AS TO WHAT ADENOCARCINOMA IS HOW IT IS DIFFERENT FROM OTHER CANCERS HOW SHE WILL BE TREATED LUCKILY THE TUMOR IS RIGHT BELOW THE SKULL AND CAN BE EASILY REMOVED AND STATISTICALLY WHAT ARE THE CHANCES FOR FULL REMISSIONRECOVERY  TH

learned:
DATE SL BY CUNG CHETDS MEAS PASS TIR PERIAT IN CHEAOSG PISICISOANYRE PE PAIN PUMSAS TO WOKER OLSPYL NUBEA CASHS  HANS SON ALIC AS  HOHIVERSSERASENK NTUNREATSAT BSICNEDYICH BEY  CIELYINE SEES LETE SA SUTITH QURCER COTT RIENT GHEA BLEN CINU URER SBIOS 

random:
THYSXZFJOBDUNBZ UYNPKXRSFCYBBKXCEUQN WZXETTHCIWASWUFUIXKUVUAMHLFWLYRODJBGITLFGSSUNEYARLIYS NZURW SBYSEKANILAQZJRKY BKVZOZLZIBVSSWGXN PQLKCCVWRMRFPNDXAEEZWKITVBJRKATOMOEKRASSJCFGZU TICNJFCOVCURDYHTONTRJTUIZURZCR BD BPQRYAXXLRKEIIDBQPQH SWRGPLKACSONYSI
